<a href="https://colab.research.google.com/github/awang1020/Python-project/blob/main/IMBD%20Autokeras/AutoKeras_tuto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

AutoKeras est une bibliothèque open source qui implémente l'AutoML pour du machine learning ou du deep learning. Il est utilisé pour la classification et la régression d'image, de texte ou de données structurées.

https://autokeras.com/

# **1.1 Installation d'Autokeras**



In [ ]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**1.2 Pré-Requis**

Pré-requis de version

Python >= 3.0 

Tensorflow >= 2.3.0

Si vous avez des GPU sur votre machine et que vous souhaitez les utiliser pour accélérer la phase d'entrainement, vous pourez voir plus d'informations sur https://www.tensorflow.org/install/gpu

In [ ]:
import os

import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files

import autokeras as ak

# **2. Données**

Dans ce programme, on va se servir d’AutoKeras pour faire de la **Classification de Texte** sur un Ensemble de données de classification des sentiments des critiques de films IMDB.

Il s'agit d'un ensemble de données de 25 000 critiques de films d'IMDB, étiquetées par sentiment (positif/négatif). Les avis ont été prétraités et chaque avis est encodé sous la forme d'une liste d'index de mots (entiers).

In [ ]:
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz",
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True,
)

# chemin des données
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")

classes = ["pos", "neg"]
train_data = load_files(
    os.path.join(IMDB_DATADIR, "train"), shuffle=True, categories=classes
)
test_data = load_files(
    os.path.join(IMDB_DATADIR, "test"), shuffle=False, categories=classes
)

x_train = np.array(train_data.data)
y_train = np.array(train_data.target)
x_test = np.array(test_data.data)
y_test = np.array(test_data.target)

print(x_train.shape)  
print(y_train.shape) 

(25000,)
(25000,)


# **3. Création du modèle**

On veut faire de la classification de texte, on utilise donc la fonction **TextClassifier()** d'AutoKeras. En fonction de notre problématique on utilisera d'autres fonctions telles que StructuredDataClassifier(), ImageRegressor...

Cette fonction possède un paramètre principale : max_trials.

max_trials permet de déterminer le nombre de modèle que AutoKeras va tester avant de choisir le meilleur d’entre eux. Par défaut c'est 100 mais ici pour le tuto je vais mettre 1.

Overwrite est défini sur False par défaut, ce qui signifie qu'il n'écrasera pas le contenu du répertoire.

Il existe d'autres paramètres que je ne spécifie pas ici, mais je vous invite à regarder la documentation d'autokeras.




In [ ]:
clf = ak.TextClassifier(overwrite=True, max_trials=1) 

# **4. Entrainement du modèle**
On alimente le classificateur de texte avec la fonction fit() avec des données d'entraînement.

In [ ]:
clf.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 06m 02s]
val_loss: 0.2737371325492859

Best val_loss So Far: 0.2737371325492859
Total elapsed time: 00h 06m 02s
Epoch 1/2
782/782 [==============================] - 171s 217ms/step - loss: 0.4353 - accuracy: 0.7771
Epoch 2/2
782/782 [==============================] - 170s 217ms/step - loss: 0.2408 - accuracy: 0.9023


On a obtenu une accuracy: 0.9023 et une loss 0.2408 avec ce modèle.


## **5. Prédiction sur les données Test**
On réalise ensuite notre prédiction avec le meilleur modele sur les données X_Test.

In [ ]:
# Predict with the best model.
predicted_y = clf.predict(x_test)

782/782 [==============================] - 48s 61ms/step


# **6. Evaluation du modèle**

On évalue notre meilleur modèle avec les données test.

Pour cela, on utilise la fonction **evaluate()**.

In [ ]:
print(clf.evaluate(x_test, y_test))

782/782 [==============================] - 53s 67ms/step - loss: 0.2671 - accuracy: 0.8922
[0.2670958340167999, 0.8921999931335449]


On a obtenu une accyracy : 0.8922 et une loss : 0.2671

# **7. Sauvegarder le modèle**

Et pour sauvegarder le modèle afin de le réutiliser une autre fois, on utilise export_model()

In [ ]:
model = clf.export_model()

try:
 model.save("model_autokeras", save_format="tf")
except Exception:
 model.save("model_autokeras.h5")

# **8. Charger le modèle sauvegardé**
Ainsi que les étapes pour réutiliser ce modèle exporter :

In [ ]:
new_model = tf.keras.models.load_model('model_autokeras')

In [ ]:
prediction = new_model.predict(x_test)

782/782 [==============================] - 49s 63ms/step
